In [10]:
import sys 
sys.path.append('..')
from cox.utils import Parameters
from cox.store import Store
from cox.readers import CollectionReader
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import math
import numpy as np
import torch as ch
from torch import Tensor
from torch.distributions.normal import Normal
from torch.distributions.multivariate_normal import MultivariateNormal
from torch.distributions import Uniform
from torch.utils.data import DataLoader
from delphi.stats.truncated_regression import truncated_regression
from delphi.utils.datasets import TruncatedRegressionDataset 
from delphi.oracle import Left, Interval
import datetime

In [37]:
STORE_PATH = '/home/pstefanou/Gradient'
STORE_TABLE_NAME = 'grad'

store = Store(STORE_PATH)

store.add_table(STORE_TABLE_NAME, { 
    'known_param_mse': float,
    'unknown_param_mse': float, 
    'ols_param_mse': float,
    'alpha': float, 
    'var': float, 
})

Logging in: /home/pstefanou/Gradient/0ba84049-902d-4672-b497-1070bb5b08ef


Experiment Constants

In [38]:
# regression parameters
NUM_SAMPLES, DIMS = 10000, 10
LOWER, UPPER = -1, 1
PREDICATE = Left(Tensor([1.0]))

In [ ]:
# begin experiment
start = datetime.datetime.now()

# perform each experiment a total of 10 times
for iter_ in range(100):        
    for VAR in range(1, 21):
        # generate data
        W = Uniform(LOWER, UPPER).sample(ch.Size([DIMS, 1]))
        W0 = Uniform(LOWER, UPPER).sample(ch.Size([1, 1]))
        X = Uniform(0, 1).sample(ch.Size([NUM_SAMPLES, DIMS]))
        y = X.mm(W) + W0 + Normal(ch.zeros(1), ch.sqrt(Tensor([VAR]))).sample(ch.Size([NUM_SAMPLES]))
        # truncate
        indices = PREDICATE(y).nonzero(as_tuple=False).flatten()
        y_trunc, x_trunc = y[indices], X[indices]

        # empirical linear regression
        reg = LinearRegression() 
        reg.fit(x_trunc, y_trunc)
        emp_var = ch.var(ch.from_numpy(reg.predict(x_trunc)) - y_trunc, dim=0).unsqueeze(0)

        # experiment parameters
        args = Parameters({ 
            'alpha': Tensor([y_trunc.size(0)/NUM_SAMPLES]), 
            'phi': PREDICATE, 
            'epochs': 10,
            'num_workers': 10, 
            'batch_size': 100,
            'bias': True,
            'num_samples': 100,
            'clamp': True, 
            'radius': 5.0, 
            'var_lr': 1e-2,
            'var': emp_var,
            'lr': 1e-1,
            'shuffle': True, 
            'tol': 5e-2,
        })


        # truncated linear regression with known noise variance
        data = TruncatedRegressionDataset(x_trunc, y_trunc, bias=args.bias, unknown=False) # dataset 
        S = DataLoader(data, batch_size=args.batch_size, shuffle=args.shuffle)
        trunc_reg = truncated_regression(phi=args.phi, alpha=args.alpha, epochs=args.epochs, var_lr=args.var_lr, lr=args.lr, num_samples=args.num_samples, var=args.var, device=args.device, tol=args.tol)
        results = trunc_reg.fit(S)
        w_, w0_ = results.weight.detach().cpu(), results.bias.detach().cpu()

        # truncated linear regression with unknown noise variance
        data = TruncatedRegressionDataset(x_trunc, y_trunc, bias=args.bias, unknown=True) # dataset 
        S = DataLoader(data, batch_size=args.batch_size, shuffle=args.shuffle)
        trunc_reg = truncated_regression(phi=args.phi, alpha=args.alpha, epochs=args.epochs, var_lr=args.var_lr, lr=args.lr, num_samples=args.num_samples, device=args.device, tol=args.tol)
        results = trunc_reg.fit(S)
        var_ = results.lambda_.inverse().detach().cpu()
        w, w0 = (results.v.detach()*var_).cpu(), (results.bias.detach()*var_).cpu()

        # calculate metrics 
        real_params, ols_params, unknown_params, known_params = ch.cat([W, W0]), ch.cat([data.w, data.w0.unsqueeze(0)]), ch.cat([w, w0]), ch.cat([w_.t(), w0_.unsqueeze(0)])
        unknown_param_mse, ols_param_mse, known_param_mse = mean_squared_error(unknown_params, real_params), mean_squared_error(ols_params, real_params), mean_squared_error(known_params, real_params)

        store[STORE_TABLE_NAME].append_row({ 
            'known_param_mse': known_param_mse,
            'unknown_param_mse': unknown_param_mse,
            'ols_param_mse': ols_param_mse,
            'alpha': float(args.alpha.flatten()),
            'var': float(VAR), 
        })

# end experiment
end = datetime.datetime.now()
print("time delta: {}".format(end - start))

# close store
store.close()

Epoch:1 | Score tensor([0.1489, 0.1182, 0.1351, 0.1509, 0.0854, 0.1370, 0.1119, 0.1259, 0.1159,
        0.0883, 0.2538]) 
Epoch:1 | Score tensor([0.1489, 0.1182, 0.1351, 0.1509, 0.0854, 0.1370, 0.1119, 0.1259, 0.1159,
        0.0883, 0.2538]) 
Epoch:1 | Score tensor([0.1101, 0.0756, 0.0897, 0.1060, 0.0777, 0.0911, 0.0927, 0.0974, 0.0821,
        0.0771, 0.1992]) 
Epoch:1 | Score tensor([0.1101, 0.0756, 0.0897, 0.1060, 0.0777, 0.0911, 0.0927, 0.0974, 0.0821,
        0.0771, 0.1992]) 
Epoch:1 | Score tensor([0.0967, 0.0842, 0.0818, 0.0912, 0.0745, 0.0868, 0.0876, 0.0840, 0.0917,
        0.0817, 0.1835]) 
Epoch:1 | Score tensor([0.0967, 0.0842, 0.0818, 0.0912, 0.0745, 0.0868, 0.0876, 0.0840, 0.0917,
        0.0817, 0.1835]) 
Epoch:1 | Score tensor([0.0919, 0.0832, 0.0858, 0.0884, 0.0673, 0.0824, 0.0856, 0.0784, 0.0887,
        0.0788, 0.1794]) 
Epoch:1 | Score tensor([0.0919, 0.0832, 0.0858, 0.0884, 0.0673, 0.0824, 0.0856, 0.0784, 0.0887,
        0.0788, 0.1794]) 
Epoch:1 | Score tensor([

Epoch:1 | Score tensor([0.0501, 0.0646, 0.0551, 0.0530, 0.0578, 0.0567, 0.0552, 0.0524, 0.0608,
        0.0535, 0.1137]) 
Epoch:1 | Score tensor([0.0503, 0.0637, 0.0553, 0.0517, 0.0565, 0.0569, 0.0547, 0.0518, 0.0597,
        0.0522, 0.1121]) 
Epoch:1 | Score tensor([0.0503, 0.0637, 0.0553, 0.0517, 0.0565, 0.0569, 0.0547, 0.0518, 0.0597,
        0.0522, 0.1121]) 
Epoch:1 | Score tensor([0.0486, 0.0627, 0.0537, 0.0504, 0.0544, 0.0545, 0.0532, 0.0508, 0.0590,
        0.0506, 0.1086]) 
Epoch:1 | Score tensor([0.0486, 0.0627, 0.0537, 0.0504, 0.0544, 0.0545, 0.0532, 0.0508, 0.0590,
        0.0506, 0.1086]) 
Epoch:1 | Score tensor([0.0487, 0.0629, 0.0548, 0.0524, 0.0546, 0.0550, 0.0545, 0.0524, 0.0604,
        0.0514, 0.1100]) 
Epoch:1 | Score tensor([0.0487, 0.0629, 0.0548, 0.0524, 0.0546, 0.0550, 0.0545, 0.0524, 0.0604,
        0.0514, 0.1100]) 
Epoch:1 | Score tensor([0.0483, 0.0635, 0.0549, 0.0525, 0.0550, 0.0560, 0.0541, 0.0533, 0.0612,
        0.0521, 0.1107]) 
Epoch:1 | Score tensor([

Epoch:2 | Score tensor([0.0464, 0.0480, 0.0285, 0.0484, 0.0296, 0.0439, 0.0584, 0.0527, 0.0389,
        0.0416, 0.0894]) 
Epoch:2 | Score tensor([0.0387, 0.0475, 0.0302, 0.0425, 0.0385, 0.0396, 0.0572, 0.0443, 0.0397,
        0.0364, 0.0865]) 
Epoch:2 | Score tensor([0.0387, 0.0475, 0.0302, 0.0425, 0.0385, 0.0396, 0.0572, 0.0443, 0.0397,
        0.0364, 0.0865]) 
Epoch:2 | Score tensor([0.0237, 0.0459, 0.0310, 0.0420, 0.0411, 0.0271, 0.0477, 0.0436, 0.0306,
        0.0376, 0.0765]) 
Epoch:2 | Score tensor([0.0237, 0.0459, 0.0310, 0.0420, 0.0411, 0.0271, 0.0477, 0.0436, 0.0306,
        0.0376, 0.0765]) 
Epoch:2 | Score tensor([0.0138, 0.0438, 0.0226, 0.0353, 0.0360, 0.0299, 0.0426, 0.0375, 0.0279,
        0.0303, 0.0637]) 
Epoch:2 | Score tensor([0.0138, 0.0438, 0.0226, 0.0353, 0.0360, 0.0299, 0.0426, 0.0375, 0.0279,
        0.0303, 0.0637]) 
Epoch:2 | Score tensor([0.0048, 0.0316, 0.0132, 0.0271, 0.0304, 0.0232, 0.0349, 0.0247, 0.0276,
        0.0180, 0.0473]) 
Epoch:2 | Score tensor([

Epoch:1 | Score tensor([ 0.0678,  0.0852,  0.0779,  0.0813,  0.0762,  0.0780,  0.0849,  0.0715,
         0.0799,  0.0706,  0.1514, -0.2288]) 
Epoch:1 | Score tensor([ 0.0678,  0.0852,  0.0779,  0.0813,  0.0762,  0.0780,  0.0849,  0.0715,
         0.0799,  0.0706,  0.1514, -0.2288]) 
Epoch:1 | Score tensor([ 0.0658,  0.0821,  0.0747,  0.0779,  0.0723,  0.0731,  0.0819,  0.0690,
         0.0767,  0.0695,  0.1467, -0.2196]) 
Epoch:1 | Score tensor([ 0.0658,  0.0821,  0.0747,  0.0779,  0.0723,  0.0731,  0.0819,  0.0690,
         0.0767,  0.0695,  0.1467, -0.2196]) 
Epoch:1 | Score tensor([ 0.0656,  0.0839,  0.0754,  0.0801,  0.0738,  0.0756,  0.0813,  0.0718,
         0.0771,  0.0708,  0.1494, -0.2243]) 
Epoch:1 | Score tensor([ 0.0656,  0.0839,  0.0754,  0.0801,  0.0738,  0.0756,  0.0813,  0.0718,
         0.0771,  0.0708,  0.1494, -0.2243]) 
Epoch:1 | Score tensor([ 0.0651,  0.0831,  0.0738,  0.0799,  0.0728,  0.0749,  0.0810,  0.0710,
         0.0762,  0.0696,  0.1474, -0.2201]) 
Epoch:

Epoch:1 | Score tensor([ 0.0572,  0.0658,  0.0598,  0.0643,  0.0577,  0.0620,  0.0650,  0.0618,
         0.0649,  0.0569,  0.1236, -0.1767]) 
Epoch:1 | Score tensor([ 0.0585,  0.0675,  0.0606,  0.0654,  0.0593,  0.0628,  0.0648,  0.0621,
         0.0652,  0.0585,  0.1259, -0.1836]) 
Epoch:1 | Score tensor([ 0.0585,  0.0675,  0.0606,  0.0654,  0.0593,  0.0628,  0.0648,  0.0621,
         0.0652,  0.0585,  0.1259, -0.1836]) 
Epoch:1 | Score tensor([ 0.0599,  0.0684,  0.0629,  0.0658,  0.0602,  0.0640,  0.0648,  0.0624,
         0.0658,  0.0598,  0.1280, -0.1881]) 
Epoch:1 | Score tensor([ 0.0599,  0.0684,  0.0629,  0.0658,  0.0602,  0.0640,  0.0648,  0.0624,
         0.0658,  0.0598,  0.1280, -0.1881]) 
Epoch:1 | Score tensor([ 0.0599,  0.0684,  0.0629,  0.0658,  0.0602,  0.0640,  0.0648,  0.0624,
         0.0658,  0.0598,  0.1280, -0.1881]) 
Epoch:1 | Score tensor([ 0.0600,  0.0685,  0.0628,  0.0651,  0.0608,  0.0638,  0.0648,  0.0625,
         0.0659,  0.0603,  0.1280, -0.1891]) 
Epoch:

...

Epoch:2 | Score tensor([0.0769, 0.0669, 0.0819, 0.0727, 0.0886, 0.0715, 0.0970, 0.0811, 0.0942,
        0.0789, 0.1706]) 
Epoch:2 | Score tensor([0.0743, 0.0677, 0.0853, 0.0752, 0.0880, 0.0672, 0.0938, 0.0781, 0.0882,
        0.0766, 0.1668]) 
Epoch:2 | Score tensor([0.0743, 0.0677, 0.0853, 0.0752, 0.0880, 0.0672, 0.0938, 0.0781, 0.0882,
        0.0766, 0.1668]) 
Epoch:2 | Score tensor([0.0772, 0.0665, 0.0842, 0.0730, 0.0836, 0.0692, 0.0884, 0.0787, 0.0855,
        0.0763, 0.1636]) 
Epoch:2 | Score tensor([0.0772, 0.0665, 0.0842, 0.0730, 0.0836, 0.0692, 0.0884, 0.0787, 0.0855,
        0.0763, 0.1636]) 
Epoch:2 | Score tensor([0.0712, 0.0669, 0.0833, 0.0708, 0.0756, 0.0708, 0.0847, 0.0765, 0.0824,
        0.0759, 0.1559]) 
Epoch:2 | Score tensor([0.0712, 0.0669, 0.0833, 0.0708, 0.0756, 0.0708, 0.0847, 0.0765, 0.0824,
        0.0759, 0.1559]) 
Epoch:2 | Score tensor([0.0712, 0.0669, 0.0833, 0.0708, 0.0756, 0.0708, 0.0847, 0.0765, 0.0824,
        0.0759, 0.1559]) 
Epoch:2 | Score tensor([

Epoch:1 | Score tensor([ 0.2755,  0.2387,  0.2304,  0.2570,  0.2605,  0.2083,  0.2705,  0.2306,
         0.1915,  0.2260,  0.4788, -1.1188]) 
Epoch:1 | Score tensor([ 0.2755,  0.2387,  0.2304,  0.2570,  0.2605,  0.2083,  0.2705,  0.2306,
         0.1915,  0.2260,  0.4788, -1.1188]) 
Epoch:1 | Score tensor([ 0.2818,  0.2491,  0.2376,  0.2683,  0.2686,  0.2102,  0.2739,  0.2468,
         0.2158,  0.2239,  0.4989, -1.2236]) 
Epoch:1 | Score tensor([ 0.2818,  0.2491,  0.2376,  0.2683,  0.2686,  0.2102,  0.2739,  0.2468,
         0.2158,  0.2239,  0.4989, -1.2236]) 
Epoch:1 | Score tensor([ 0.2858,  0.2483,  0.2422,  0.2687,  0.2758,  0.2201,  0.2641,  0.2536,
         0.2187,  0.2253,  0.5012, -1.1782]) 
Epoch:1 | Score tensor([ 0.2858,  0.2483,  0.2422,  0.2687,  0.2758,  0.2201,  0.2641,  0.2536,
         0.2187,  0.2253,  0.5012, -1.1782]) 
Epoch:1 | Score tensor([ 0.2932,  0.2584,  0.2479,  0.2702,  0.2730,  0.2264,  0.2683,  0.2606,
         0.2294,  0.2347,  0.5120, -1.1819]) 
Epoch:

Epoch:2 | Score tensor([0.0413, 0.0406, 0.0515, 0.0253, 0.0285, 0.0630, 0.0553, 0.0426, 0.0677,
        0.0621, 0.0998, 0.3379]) 
Epoch:2 | Score tensor([0.0413, 0.0406, 0.0515, 0.0253, 0.0285, 0.0630, 0.0553, 0.0426, 0.0677,
        0.0621, 0.0998, 0.3379]) 
Epoch:2 | Score tensor([0.0563, 0.0565, 0.0609, 0.0443, 0.0656, 0.0862, 0.0690, 0.0724, 0.0886,
        0.0891, 0.1450, 0.1092]) 
Epoch:2 | Score tensor([0.0563, 0.0565, 0.0609, 0.0443, 0.0656, 0.0862, 0.0690, 0.0724, 0.0886,
        0.0891, 0.1450, 0.1092]) 
Epoch:2 | Score tensor([0.0471, 0.0485, 0.0314, 0.0199, 0.0459, 0.0527, 0.0452, 0.0557, 0.0666,
        0.0674, 0.1016, 0.3964]) 
Epoch:2 | Score tensor([0.0471, 0.0485, 0.0314, 0.0199, 0.0459, 0.0527, 0.0452, 0.0557, 0.0666,
        0.0674, 0.1016, 0.3964]) 
Epoch:2 | Score tensor([0.0494, 0.0468, 0.0425, 0.0274, 0.0558, 0.0531, 0.0468, 0.0645, 0.0686,
        0.0765, 0.1089, 0.3505]) 
Epoch:2 | Score tensor([0.0494, 0.0468, 0.0425, 0.0274, 0.0558, 0.0531, 0.0468, 0.0645, 0.

Epoch:3 | Score tensor([-0.0030, -0.0858, -0.0350, -0.0271, -0.0360, -0.0571, -0.0598, -0.1020,
        -0.0336, -0.0405, -0.0564,  0.3306]) 
Epoch:3 | Score tensor([-0.0030, -0.0858, -0.0350, -0.0271, -0.0360, -0.0571, -0.0598, -0.1020,
        -0.0336, -0.0405, -0.0564,  0.3306]) 
Epoch:3 | Score tensor([-0.0030, -0.0858, -0.0350, -0.0271, -0.0360, -0.0571, -0.0598, -0.1020,
        -0.0336, -0.0405, -0.0564,  0.3306]) 
Epoch:3 | Score tensor([-0.0151, -0.0761, -0.0230, -0.0363, -0.0165, -0.0525, -0.0567, -0.0833,
        -0.0188, -0.0330, -0.0343,  0.2147]) 
Epoch:3 | Score tensor([-0.0151, -0.0761, -0.0230, -0.0363, -0.0165, -0.0525, -0.0567, -0.0833,
        -0.0188, -0.0330, -0.0343,  0.2147]) 
Epoch:3 | Score tensor([-0.0422, -0.0897, -0.0595, -0.0531, -0.0543, -0.0713, -0.0708, -0.0989,
        -0.0429, -0.0464, -0.0844,  0.4777]) 
Epoch:3 | Score tensor([-0.0422, -0.0897, -0.0595, -0.0531, -0.0543, -0.0713, -0.0708, -0.0989,
        -0.0429, -0.0464, -0.0844,  0.4777]) 
Epoch:

Epoch:1 | Score tensor([0.2733, 0.2685, 0.2571, 0.2925, 0.2854, 0.2937, 0.2966, 0.2669, 0.2673,
        0.3010, 0.5595]) 
Epoch:1 | Score tensor([0.2733, 0.2685, 0.2571, 0.2925, 0.2854, 0.2937, 0.2966, 0.2669, 0.2673,
        0.3010, 0.5595]) 
Epoch:1 | Score tensor([0.2733, 0.2685, 0.2571, 0.2925, 0.2854, 0.2937, 0.2966, 0.2669, 0.2673,
        0.3010, 0.5595]) 
Epoch:1 | Score tensor([0.2755, 0.2676, 0.2589, 0.2963, 0.2834, 0.2895, 0.2948, 0.2677, 0.2652,
        0.2993, 0.5594]) 
Epoch:1 | Score tensor([0.2755, 0.2676, 0.2589, 0.2963, 0.2834, 0.2895, 0.2948, 0.2677, 0.2652,
        0.2993, 0.5594]) 
Epoch:1 | Score tensor([0.2667, 0.2621, 0.2531, 0.2893, 0.2808, 0.2794, 0.2865, 0.2589, 0.2624,
        0.2924, 0.5471]) 
Epoch:1 | Score tensor([0.2667, 0.2621, 0.2531, 0.2893, 0.2808, 0.2794, 0.2865, 0.2589, 0.2624,
        0.2924, 0.5471]) 
Epoch:1 | Score tensor([0.2578, 0.2528, 0.2449, 0.2761, 0.2765, 0.2728, 0.2746, 0.2537, 0.2564,
        0.2828, 0.5316]) 
Epoch:1 | Score tensor([

In [ ]:
reader = CollectionReader(STORE_PATH)
results = reader.df(STORE_TABLE_NAME)
results.head()

In [ ]:
known, unknown, ols = results[['known_param_mse', 'var']], results[['unknown_param_mse', 'var']], results[['ols_param_mse', 'var']]

In [ ]:
# plot results for regression parameter MSE
sns.lineplot(data=unknown, x='var', y='unknown_param_mse', label='unknown', color='blue')
sns.lineplot(data=known, x='var', y='known_param_mse', label='known', color='purple')
ax = sns.lineplot(data=ols, x='var', y='ols_param_mse', label='ols', color='red')
ax.set(xlabel='Noise Variance', ylabel='Squared Error')
plt.show()
# close reader
reader.close()

In [22]:
reader = CollectionReader('/home/pstefanou/Uniform2/')
results = reader.df('uniform')

 60%|██████    | 3/5 [00:00<00:00, 187.94it/s]


OSError: HDF5 error back trace

  File "H5F.c", line 509, in H5Fopen
    unable to open file
  File "H5Fint.c", line 1400, in H5F__open
    unable to open file
  File "H5Fint.c", line 1615, in H5F_open
    unable to lock the file
  File "H5FD.c", line 1640, in H5FD_lock
    driver lock request failed
  File "H5FDsec2.c", line 941, in H5FD_sec2_lock
    unable to lock file, errno = 11, error message = 'Resource temporarily unavailable'

End of HDF5 error back trace

Unable to open/create file '/home/pstefanou/Uniform2/9eb85ee0-dcf6-4c9a-a1b7-94cc3558ba8d/store.h5'

In [23]:
results

,known_param_mse,unknown_param_mse,ols_param_mse,alpha,var,exp_id
0,0.229495,0.229495,0.229495,0.3954,1.0,6c5a334f-b3f1-4d57-81f2-4d59dd51ef80
1,0.168892,0.167581,0.168892,0.7402,2.0,6c5a334f-b3f1-4d57-81f2-4d59dd51ef80
2,0.650567,0.676718,0.678642,0.0641,3.0,6c5a334f-b3f1-4d57-81f2-4d59dd51ef80
3,0.895550,0.874560,0.895550,0.2629,4.0,6c5a334f-b3f1-4d57-81f2-4d59dd51ef80
4,0.357972,0.403891,0.385633,0.4885,5.0,6c5a334f-b3f1-4d57-81f2-4d59dd51ef80
5,0.835289,0.924343,0.835289,0.6530,6.0,6c5a334f-b3f1-4d57-81f2-4d59dd51ef80
6,0.452985,0.489863,0.493153,0.4877,7.0,6c5a334f-b3f1-4d57-81f2-4d59dd51ef80
7,1.088347,1.136333,1.152218,0.3656,8.0,6c5a334f-b3f1-4d57-81f2-4d59dd51ef80
8,1.193646,0.996717,1.282450,0.3545,9.0,6c5a334f-b3f1-4d57-81f2-4d59dd51ef80
9,0.926922,255.613586,0.972060,0.4802,10.0,6c5a334f-b3f1-4d57-81f2-4d59dd51ef80


In [21]:
reader.close()

ClosedFileError: /home/pstefanou/Uniform2/a5110297-3b99-46a1-a6fd-3c436d7caf34/store.h5 file is not open!